In [2]:
from IPython.display import display
import os

# 上傳檔案（會跳出檔案選擇器）
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

In [3]:
import os
from pathlib import Path

# 假設你只上傳了一個檔案
# Check if any file is uploaded and get the filename
if upload.value:
    # Get the first (and likely only) filename from the dictionary keys
    filename = list(upload.value.keys())[0]
    fileinfo = upload.value[filename] # Access the file info dictionary using the filename as key
else:
    print("No file uploaded.")
    # Handle the case where no file is uploaded, perhaps by exiting or prompting the user.
    # For this example, we'll assume a file was uploaded as per the traceback context.
    raise FileNotFoundError("No file was uploaded.")


# 顯示內容結構（除錯用）
print(fileinfo)

# 儲存 kaggle.json
# filename is already obtained above
content = fileinfo['content']

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "wb") as f:
    f.write(content)

# 設定權限（Linux/macOS 建議）
os.chmod(kaggle_json_path, 0o600)

print(f"{filename} 已成功儲存至 {kaggle_json_path}")

{'metadata': {'name': 'kaggle.json', 'type': 'application/json', 'size': 64, 'lastModified': 1745215961986}, 'content': b'{"username":"suchiwen","key":"01a925cee9e9e9d232008524b0434fb9"}'}
kaggle.json 已成功儲存至 /root/.kaggle/kaggle.json


In [4]:
!kaggle datasets list -s cifar

ref                                                     title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
fedesoriano/cifar100                                    CIFAR-100 Python                                  168517809  2020-12-26 08:37:10.143000          12303        176  1.0              
pankrzysiu/cifar10-python                               CIFAR-10 Python                                   340613496  2018-01-27 13:42:40.967000          14829        252  0.75             
petitbonney/cifar10-image-recognition                   CIFAR-10                                         1007971063  2019-10-01 12:50:23.227000           2890         27  0.8235294        
valentynsichkar/cifar10-preprocessed                   

In [5]:
!pip install -U kaggle
!pip install --upgrade pandas
import os
import zipfile

# 建立 Kaggle 資料夾
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 下載 Dog Breed Identification 資料集
!kaggle competitions download -c dog-breed-identification --force
!unzip -oq dog-breed-identification.zip -d dog-breed-identification


import pandas as pd
import numpy as np
labels = pd.read_csv('dog-breed-identification/labels.csv')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
cp: cannot stat 'kaggle.json': No such file or directory
 98% 675M/691M [00:03<00:00, 115MB/s] 
100% 691M/691M [00:03<00:00, 196MB/s]


In [19]:
# ========== 訓練設定 ==========
import torch
from torch.utils.data import random_split, DataLoader, WeightedRandomSampler
from collections import Counter
from torchvision import transforms
from PIL import Image
import os
import pandas as pd # Ensure pandas is imported if not already
import numpy as np # Ensure numpy is imported if not already
import shutil # Ensure shutil is imported if not already

# Assuming DogBreedImageFolder class and DogCNN model are defined in previous cells and working correctly.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomAffine(15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset & Split
# Ensure the 'dog_images/train' directory is correctly populated by previous steps
dataset = DogBreedImageFolder("dog_images/train", transform=train_transform)
val_ratio = 0.2
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform  # 替換驗證用 transform

# Weighted Sampler
train_indices = train_dataset.indices
# Access the labels from the original dataset using the training indices
train_labels = [dataset.labels[i] for i in train_indices]
label_counts = Counter(train_labels)

# Calculate sample weights for the WeightedRandomSampler
sample_weights = [1.0 / label_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# Calculate class weights for the loss function (if needed, e.g., for CrossEntropyLoss or FocalLoss)
# This is based on the counts in the training subset
num_classes = len(dataset.class_names) # Get the number of classes from the dataset
class_sample_counts = [label_counts.get(i, 0) for i in range(num_classes)] # Use 0 for classes not present in train split initially
# Avoid division by zero for classes not in the training split (though rare in this case)
weights = torch.tensor([1.0 / (c if c > 0 else 1.0) for c in class_sample_counts], dtype=torch.float).to(device)


train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Print label_counts and weights for verification
print("Label counts in training set:", label_counts)
print("Class weights tensor for loss:", weights)

# Note: The earlier code block that caused the error and recalculated label_counts and class_weights
# should be removed or commented out as it was redundant and incorrect in its placement.
# The calculation of weights for the FocalLoss is correctly placed later in the notebook.

Label counts in training set: Counter({97: 106, 73: 98, 87: 95, 42: 95, 11: 93, 52: 92, 100: 90, 1: 88, 7: 88, 69: 87, 109: 85, 6: 85, 3: 83, 68: 83, 26: 83, 61: 82, 80: 82, 13: 81, 59: 80, 94: 79, 9: 78, 93: 77, 75: 76, 84: 76, 101: 76, 70: 75, 30: 75, 98: 75, 55: 74, 114: 74, 86: 74, 117: 74, 88: 73, 35: 73, 17: 73, 8: 72, 14: 72, 78: 72, 28: 71, 40: 70, 90: 70, 95: 70, 31: 70, 33: 70, 118: 70, 119: 69, 2: 69, 102: 69, 82: 69, 60: 69, 79: 68, 71: 67, 56: 67, 20: 67, 15: 66, 92: 66, 39: 66, 96: 65, 53: 65, 62: 65, 19: 65, 37: 65, 89: 65, 67: 65, 0: 65, 57: 64, 27: 64, 32: 64, 10: 64, 107: 64, 105: 64, 25: 64, 111: 64, 36: 63, 54: 63, 110: 63, 116: 63, 104: 63, 58: 63, 91: 63, 81: 62, 21: 62, 99: 61, 44: 61, 76: 61, 38: 61, 50: 61, 64: 61, 74: 61, 12: 61, 29: 61, 115: 61, 18: 60, 85: 60, 34: 60, 4: 59, 108: 59, 77: 59, 63: 59, 45: 58, 51: 58, 112: 58, 47: 58, 5: 57, 24: 57, 66: 57, 65: 56, 103: 56, 113: 56, 48: 55, 83: 55, 46: 55, 41: 54, 49: 54, 106: 54, 23: 54, 43: 52, 72: 52, 16: 52

In [22]:
#Dataset 建立
import os
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from collections import Counter
from torch.utils.data import WeightedRandomSampler

# Assuming the 'labels' dataframe is already loaded from labels.csv in a previous cell
# Example (if labels was not loaded previously, add this):
# labels = pd.read_csv('dog-breed-identification/labels.csv')


class DogBreedDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        # Assuming 'label' column in dataframe contains string labels
        # We need to map these to integers for training
        unique_labels = sorted(self.dataframe['label'].unique())
        self.label_to_int = {label: i for i, label in enumerate(unique_labels)}
        self.int_to_label = {i: label for label, i in self.label_to_int.items()}
        self.dataframe['label_int'] = self.dataframe['label'].map(self.label_to_int)


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_id = row['id']
        # Use the integer label for training
        label_int = row['label_int']
        # Assuming images are in 'image_dir' and named as 'id.jpg'
        img_path = os.path.join(self.image_dir, f'{image_id}.jpg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label_int

In [26]:
# ============ Step 2: 自定義 CNN 模型 ============
# ============ Step 2: 自定義 CNN 模型 ============
import torch.nn as nn
import torch

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        # Calculate the size of the flattened features
        # Input size is 224x224, after 3 MaxPool(2) layers, spatial size becomes 224 / (2*2*2) = 224 / 8 = 28
        # The number of channels is 128
        flattened_size = 128 * 28 * 28
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            # Update the input features to the first Linear layer
            nn.Linear(flattened_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Re-instantiate the model after modifying the class definition
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Ensure num_classes is correctly defined from the dataset in a previous cell
# If not defined, you might need to define it here, e.g., num_classes = 120
model = SimpleCNN(num_classes).to(device)

In [27]:
# ========== 8. 訓練主程式 ==========
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# criterion = nn.CrossEntropyLoss(weight=weights_tensor) # Original line causing error
criterion = nn.CrossEntropyLoss(weight=weights) # Use the 'weights' tensor calculated earlier
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {total_loss:.4f} | Val Acc: {val_acc:.4f}")

100%|██████████| 512/512 [00:36<00:00, 14.12it/s]


Epoch 1/10 | Train Loss: 2642.5781 | Val Acc: 0.0098


100%|██████████| 512/512 [00:36<00:00, 14.09it/s]


Epoch 2/10 | Train Loss: 2449.3411 | Val Acc: 0.0059


100%|██████████| 512/512 [00:36<00:00, 13.96it/s]


...

Epoch 8/10 | Train Loss: 2446.9847 | Val Acc: 0.0083


100%|██████████| 512/512 [00:35<00:00, 14.37it/s]


Epoch 9/10 | Train Loss: 2446.3154 | Val Acc: 0.0083


100%|██████████| 512/512 [00:36<00:00, 13.94it/s]


Epoch 10/10 | Train Loss: 2446.6181 | Val Acc: 0.0088


In [28]:

# 預測
model.eval()
all_predictions = []

with torch.no_grad():
    for images, img_ids in tqdm(test_loader):
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()  # 機率

        for img_id, prob in zip(img_ids, probs):
            all_predictions.append([img_id] + prob.tolist())

# class_names 來自 dataset.class_names
class_names = dataset.class_names

# 寫入 submission.csv
submission_df = pd.DataFrame(all_predictions, columns=['id'] + class_names)
submission_df['id'] = submission_df['id'].str.replace('.jpg', '', regex=False)  # 去掉副檔名
submission_df.to_csv("submission_cnn.csv", index=False)

print("submission_cnn.csv 已產生完成，可上傳至 Kaggle 評分。")

100%|██████████| 324/324 [00:48<00:00,  6.66it/s]


submission.csv 已產生完成，可上傳至 Kaggle 評分。
